# ResNet

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Загружаем предобученную ResNet18
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
for param in model.parameters():
    param.requires_grad = False  # замораживаем слои

# 2. Меняем выходной слой
model.fc = nn.Linear(model.fc.in_features, 2)  # кошка/собака
model = model.to(device)

# 3. Загружаем свои данные
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
dataset = ImageFolder("data/pets", transform=transform)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# 4. Обучаем только новый слой
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

for epoch in range(2):
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} complete!")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 166MB/s]


FileNotFoundError: [Errno 2] No such file or directory: 'data/pets'

In [ ]:
# 5. Функция для предсказания
def predict_image(image_path, model, class_names=['cat', 'dog']):
    """
    Предсказывает класс для одного изображения
    """
    # Переводим модель в режим оценки
    model.eval()

    # Загружаем и преобразуем изображение
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)  # добавляем batch dimension

    # Делаем предсказание
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][predicted_class].item()

    return class_names[predicted_class], confidence

# 6. Пример использования
# Предположим, у вас есть изображение 'test_image.jpg'
try:
    predicted_class, confidence = predict_image('test_image.jpg', model)
    print(f"Предсказанный класс: {predicted_class}")
    print(f"Уверенность: {confidence:.2%}")
except FileNotFoundError:
    print("Файл 'test_image.jpg' не найден")

# 7. Предсказание для нескольких изображений из датасета
def predict_dataloader(model, dataloader, class_names=['cat', 'dog']):
    """
    Предсказывает классы для всего даталоадера
    """
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Точность на тестовых данных: {accuracy:.2f}%')

# Использование
predict_dataloader(model, loader)

# 8. Сохранение модели для последующего использования
torch.save(model.state_dict(), 'pet_classifier.pth')
print("Модель сохранена как 'pet_classifier.pth'")

# 9. Загрузка модели для предсказаний (если нужно использовать позже)
def load_model(model_path):
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model.to(device)

# loaded_model = load_model('pet_classifier.pth')

# Bert

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Загружаем готовую модель
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Загружаем датасет (пример — отзывы IMDB)
dataset = load_dataset("imdb", split="train[:100%]").train_test_split(test_size=0.2)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

dataset = dataset.map(tokenize, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
# Сохраняем модель
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")


In [ ]:

# Функция для предсказания
def predict_sentiment(text, model, tokenizer):
    """
    Предсказывает sentiment текста (0 - негативный, 1 - позитивный)
    """
    model.eval()

    # Токенизируем текст
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Делаем предсказание
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=1).item()
        confidence = predictions[0][predicted_class].item()

    sentiment = "позитивный" if predicted_class == 1 else "негативный"
    return sentiment, confidence

# Пример предсказания
try:
    test_text = "This movie was absolutely fantastic! I loved every minute of it."
    sentiment, confidence = predict_sentiment(test_text, model, tokenizer)
    print(f"Текст: '{test_text}'")
    print(f"Предсказанный sentiment: {sentiment}")
    print(f"Уверенность: {confidence:.2%}")

    test_text2 = "This was the worst movie I have ever seen. Terrible acting and boring plot."
    sentiment2, confidence2 = predict_sentiment(test_text2, model, tokenizer)
    print(f"\nТекст: '{test_text2}'")
    print(f"Предсказанный sentiment: {sentiment2}")
    print(f"Уверенность: {confidence2:.2%}")

except NameError:
    print("Для предсказаний нужно импортировать torch")
    import torch
    sentiment, confidence = predict_sentiment(test_text, model, tokenizer)
    print(f"Предсказанный sentiment: {sentiment}")
    print(f"Уверенность: {confidence:.2%}")

In [ ]:
# 2. Предсказание для нескольких текстов из датасета
def predict_dataset_samples(model, tokenizer, dataset, num_samples=5):
    """
    Предсказывает sentiment для нескольких примеров из датасета
    """
    model.eval()

    print("Предсказания для примеров из датасета:")
    print("-" * 80)

    # Берем случайные примеры из тестового набора
    test_samples = dataset["test"].select(range(num_samples))

    for i in range(num_samples):
        text = test_samples[i]["text"]
        true_label = test_samples[i]["label"]
        true_sentiment = "позитивный" if true_label == 1 else "негативный"

        # Предсказываем
        pred_sentiment, confidence, pred_class = predict_sentiment(text, model, tokenizer)

        # Сравниваем с истинным значением
        is_correct = "✓" if pred_class == true_label else "✗"

        print(f"Пример {i+1}:")
        print(f"Текст: {text[:100]}...")  # Показываем только первые 100 символов
        print(f"Истинный sentiment: {true_sentiment}")
        print(f"Предсказанный sentiment: {pred_sentiment} {is_correct}")
        print(f"Уверенность: {confidence:.2%}")
        print("-" * 80)

# 3. Оценка точности на всем тестовом наборе
def evaluate_model(model, tokenizer, test_dataset):
    """
    Оценивает точность модели на всем тестовом наборе
    """
    model.eval()
    correct = 0
    total = 0

    # Используем меньший батч для экономии памяти
    from torch.utils.data import DataLoader
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    dataloader = DataLoader(test_dataset, batch_size=8)

    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'label'}
            labels = batch['label'].to(model.device)

            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=1)

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Точность на тестовом наборе: {accuracy:.2%}")
    return accuracy

# 4. Примеры использования

print("=== ПРЕДСКАЗАНИЕ ДЛЯ ОДНОГО ТЕКСТА ===")
test_texts = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "This was the worst movie I have ever seen. Terrible acting and boring plot.",
    "The film had some good moments but overall it was quite disappointing."
]

for i, text in enumerate(test_texts):
    sentiment, confidence, _ = predict_sentiment(text, model, tokenizer)
    print(f"Текст {i+1}: '{text}'")
    print(f"Предсказанный sentiment: {sentiment}")
    print(f"Уверенность: {confidence:.2%}\n")

print("\n=== ПРЕДСКАЗАНИЕ ДЛЯ ДАТАСЕТА ===")
# Предсказание для примеров из датасета
predict_dataset_samples(model, tokenizer, dataset, num_samples=3)

print("\n=== ОЦЕНКА ТОЧНОСТИ ===")
# Оценка точности на тестовом наборе
evaluate_model(model, tokenizer, dataset["test"])

# 5. Предсказание для новых пользовательских текстов
def predict_custom_texts():
    """
    Функция для интерактивного предсказания
    """
    print("\n=== ИНТЕРАКТИВНОЕ ПРЕДСКАЗАНИЕ ===")
    print("Введите тексты для анализа sentiment (для выхода введите 'quit'):")

    while True:
        text = input("\nВведите текст: ")
        if text.lower() == 'quit':
            break

        sentiment, confidence, _ = predict_sentiment(text, model, tokenizer)
        print(f"Результат: {sentiment} (уверенность: {confidence:.2%})")

# Раскомментируйте для интерактивного режима
# predict_custom_texts()